# Sentiment analysis of IMDB reviews

##### INSTALLATION OF NECESSARY LIBRARIES AND PACKAGES

In [3]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##### IMPORT THE NECESSARY PACKAGES

In [1]:
import tensorflow as tf

In [2]:
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



##### IMPORT THE DATASET

Two dataset for Training and Testing

In [5]:
imdb_reviews=pd.read_csv("/content/imdb_reviews.csv")
test_reviews=pd.read_csv("/content/test_reviews.csv")

Training data


In [6]:
imdb_reviews.head()

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative


Testing data

In [7]:
test_reviews.head()

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive


### PREPROCESSING THE DATA

We can not pass the string data to our model directly, so we need to transform the string data into integer format.

In [8]:
word_index=pd.read_csv("/content/word_indexes.csv")

The word index file contains mapping from words to integers.

In [9]:
word_index.head()

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411


Next we are going to convert the word_index dataframe into a python dictionary so that we can use it for converting our reviews from string to integer format.

In [10]:
word_index=dict(zip(word_index.Words,word_index.Indexes))

In [11]:
word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

Now we define a function review_encoder that encodes the reviews into integer format according to the mapping specified by word_index file.

In [12]:
def review_encoder(text):
  arr=[word_index[word] for word in text]
  return arr

We split the reviews from their corresponding sentiments so that we can preprocess the reviews and sentiments separately and then later pass it to our model.

In [13]:
train_data,train_labels=imdb_reviews['Reviews'],imdb_reviews['Sentiment']
test_data, test_labels=test_reviews['Reviews'],test_reviews['Sentiment']

Before transforming the reviews as integers we need to tokenize or split the review on the basis of whitespaces

In [14]:
train_data=train_data.apply(lambda review:review.split())
test_data=test_data.apply(lambda review:review.split())

Since we have tokenized the reviews now we can apply the review_encoder function to each review and transform the reviews into integer format.

In [15]:
train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)

In [16]:
train_data.head()

0    [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...
1    [1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...
2    [1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...
3    [1, 4, 2, 2, 33, 2804, 4, 2040, 432, 111, 153,...
4    [1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...
Name: Reviews, dtype: object

need to encode the sentiments and we are labeling the positive sentiment as 1 and negative sentiment as 0.

In [17]:
def encode_sentiments(x):
  if x=='positive':
    return 1
  else:
    return 0

train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)

In [18]:
train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)

### Building the model

Our model is a neural network and it consits of the following layers :

one word embedding layer which creates word embeddings of length 16 from integer encoded review.

second layer is global average pooling layer which is used to prevent overfitting by reducing the number of parameters.

then a dense layer which has 16 hidden units and uses relu as activation function

the final layer is the output layer which uses sigmoid as activation function

In [19]:
model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                        keras.layers.GlobalAveragePooling1D(),
                        keras.layers.Dense(16,activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')])

# compiling the model

In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Adam is used as optimization function for our model.

Binary cross entropy loss function is used as loss function for the model.

Accuracy is used as the metric for evaluating the model.

In [21]:
#training the model
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
17/17 [==============================] - 2s 52ms/step - loss: 0.6929 - accuracy: 0.5037 - val_loss: 0.6926 - val_accuracy: 0.4951
Epoch 2/30
17/17 [==============================] - 1s 41ms/step - loss: 0.6918 - accuracy: 0.5078 - val_loss: 0.6914 - val_accuracy: 0.5185
Epoch 3/30
17/17 [==============================] - 1s 41ms/step - loss: 0.6904 - accuracy: 0.5104 - val_loss: 0.6899 - val_accuracy: 0.5175
Epoch 4/30
17/17 [==============================] - 1s 41ms/step - loss: 0.6882 - accuracy: 0.6585 - val_loss: 0.6870 - val_accuracy: 0.6952
Epoch 5/30
17/17 [==============================] - 1s 41ms/step - loss: 0.6843 - accuracy: 0.7190 - val_loss: 0.6823 - val_accuracy: 0.7388
Epoch 6/30
17/17 [==============================] - 1s 41ms/step - loss: 0.6782 - accuracy: 0.7258 - val_loss: 0.6765 - val_accuracy: 0.6951
Epoch 7/30
17/17 [==============================] - 1s 41ms/step - loss: 0.6706 - accuracy: 0.7507 - val_loss: 0.6683 - val_accuracy: 0.7527
Epoch 8/30
17

In [22]:
loss,accuracy=model.evaluate(test_data,test_labels)

297/297 [==============================] - 1s 3ms/step - loss: 0.3630 - accuracy: 0.8578


 model is giving an accuracy of 88.58% on the testing data.

In [30]:
index=np.random.randint(1,1000)
user_review=test_reviews.loc[index]
print(user_review)


Reviews      <START this is one of my personal favorites a ...
Sentiment                                             positive
Name: 615, dtype: object


In [24]:
user_review=test_data[index]
user_review=np.array([user_review])
if (model.predict(user_review)>0.5).astype("int32"):
  print("positive sentiment")
else:
  print("negative sentiment")


1/1 [==============================] - 0s 83ms/step
positive sentiment


##### SENTIMENTAL ANALYSIS OF THE REVIEW IS DONE!